# 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 랜덤 시드 설정

In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 데이터 불러오기

In [3]:
BASE_PATH = r'C:\Users\com\Documents\GitHub\level2-competitiveds-recsys-06\data\etc'

train_data = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
test_data = pd.read_csv(os.path.join(BASE_PATH, 'test.csv'))
sample_submission = pd.read_csv(os.path.join(BASE_PATH, 'sample_submission.csv'))

# 데이터 EDA 수행하기

In [4]:
print("train, test_data, sample_submission shape : ", train_data.shape, test_data.shape, sample_submission.shape)
print("train data 상단 5개 정보 확인 : ", train_data.head())
print("test data 상단 5개 정보 확인 : ", test_data.head())
print("sample_submission 상단 5개 정보 확인 : ", sample_submission.head())
print('train data 변수 요약 정보 확인 : ', train_data.describe())
print("test data 변수 요약 정보 확인 : ", test_data.describe())



train, test_data, sample_submission shape :  (1801228, 11) (150172, 10) (150172, 2)
train data 상단 5개 정보 확인 :     index  area_m2  contract_year_month  contract_day  contract_type  floor  \
0      0  84.9981               201906            25              2      9   
1      1  84.9981               202003            26              2     20   
2      2  84.9981               202003            28              2      8   
3      3  59.3400               201907            15              2      1   
4      4  59.8100               201904            12              2      6   

   built_year   latitude   longitude  age  deposit  
0        2019  37.054314  127.045216    0  17000.0  
1        2019  37.054314  127.045216    1  23000.0  
2        2019  37.054314  127.045216    1  23000.0  
3        1986  36.964647  127.055847   33   5000.0  
4        1995  36.972390  127.084514   24   1800.0  
test data 상단 5개 정보 확인 :     index  area_m2  contract_year_month  contract_day  contract_type  floor  \


In [5]:
print(train_data.columns)
train_data.head()

Index(['index', 'area_m2', 'contract_year_month', 'contract_day',
       'contract_type', 'floor', 'built_year', 'latitude', 'longitude', 'age',
       'deposit'],
      dtype='object')


,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,deposit
0,0,84.9981,201906,25,2,9,2019,37.054314,127.045216,0,17000.0
1,1,84.9981,202003,26,2,20,2019,37.054314,127.045216,1,23000.0
2,2,84.9981,202003,28,2,8,2019,37.054314,127.045216,1,23000.0
3,3,59.3400,201907,15,2,1,1986,36.964647,127.055847,33,5000.0
4,4,59.8100,201904,12,2,6,1995,36.972390,127.084514,24,1800.0


In [6]:
# train_data.shape #(1594362, 8)
train_data.groupby(by=["latitude", "longitude", "built_year"]).max()

,,,index,area_m2,contract_year_month,contract_day,contract_type,floor,age,deposit
latitude,longitude,built_year,,,,,,,,
36.917910,126.908029,1996,775455,84.5900,202304,28,2,14,27,10000.0
36.957089,127.047449,1990,774308,46.9800,202311,31,2,5,33,8500.0
36.959894,127.045371,2020,162840,83.2576,202103,22,2,12,1,23000.0
36.960034,127.059939,2005,774337,84.5344,202309,31,2,13,18,19000.0
36.960936,127.054219,2010,774345,179.6433,202311,20,2,8,13,37000.0
...,...,...,...,...,...,...,...,...,...,...
38.096380,127.076702,2002,790509,60.7000,202305,30,2,6,21,6500.0
38.101890,127.079581,1996,790527,84.9500,202312,31,2,13,27,9500.0
38.104768,127.079949,2023,790615,84.9986,202312,31,2,23,0,23000.0


In [7]:
# train_data.shape #(1594362, 8)
train_data.groupby(by=["contract_type"]).count()

,index,area_m2,contract_year_month,contract_day,floor,built_year,latitude,longitude,age,deposit
contract_type,,,,,,,,,,
0,559687,559687,559687,559687,559687,559687,559687,559687,559687,559687
1,314755,314755,314755,314755,314755,314755,314755,314755,314755,314755
2,926786,926786,926786,926786,926786,926786,926786,926786,926786,926786


```python
tarin_data.shape # == (1594362, 8)
train_data.contract_year_month # min = 201904 / max = 202306
area_m2 # min = 10.3215/ max 317.36
deposit # min = 300 / 1,100,000
```
24년 지어진 건물의 최소 계약일자는 21년 9월 이며 최대 계약 일은 23년 12월이다.```python
tarin_data.shape # == (1594362, 8)
train_data.contract_year_month # min = 201904 / max = 202306
area_m2 # min = 10.3215/ max 317.36
deposit # min = 300 / 1,100,000
```
24년 지어진 건물의 최소 계약일자는 21년 9월 이며 최대 계약 일은 23년 12월이다.

# 대회에 필요한 데이터만 활용하기

In [35]:
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(""))))
from src.pre_procecss.feature_add import FeatureAddition

In [38]:
train_data["_type"] = "train"
test_data["_type"] = "test"

concat_df = pd.concat([train_data,test_data],axis=0)
df= FeatureAddition(concat_df).get_data()
df

,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,deposit,apt_idx,area,area_price,area_m2_price,_type
0,0,84.9981,201906,25,2,9,2019,37.054314,127.045216,0,17000.0,0,25.8,658.914729,200.004471,train
1,1,84.9981,202003,26,2,20,2019,37.054314,127.045216,1,23000.0,0,25.8,891.472868,270.594284,train
2,2,84.9981,202003,28,2,8,2019,37.054314,127.045216,1,23000.0,0,25.8,891.472868,270.594284,train
3,3,59.3400,201907,15,2,1,1986,36.964647,127.055847,33,5000.0,1,18.0,277.777778,84.260195,train
4,4,59.8100,201904,12,2,6,1995,36.972390,127.084514,24,1800.0,2,18.1,99.447514,30.095302,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150167,150167,115.5101,202402,27,0,17,2010,37.528394,126.659398,14,NaN,17869,35.0,NaN,NaN,test
150168,150168,142.8738,202403,2,0,4,2010,37.528394,126.659398,14,NaN,17869,43.3,NaN,NaN,test
150169,150169,142.8738,202403,16,1,13,2010,37.528394,126.659398,14,NaN,17869,43.3,NaN,NaN,test
150170,150170,114.9285,202403,22,1,2,2010,37.528394,126.659398,14,NaN,17869,34.8,NaN,NaN,test


In [46]:
# 1. 그룹별로 같은 면적을 가진 아파트 찾기
def fill_deposit_na(df):
    # 그룹별로 작업하기 위해 groupby 사용
    for group, group_df in df.groupby('apt_idx'):
        # 그룹 내에서 NaN이 있는 row에 대해 처리
        for i, row in group_df[group_df['deposit'].isna()].iterrows():
            # 2. 같은 면적을 가진 아파트의 deposit 값을 찾음
            same_area_rows = group_df[group_df['area_m2'] == row['area_m2']]

            if len(same_area_rows[same_area_rows['deposit'].notna()]) > 0:
                # 3. 같은 면적을 가진 아파트의 deposit 값에서 5% 인상된 값으로 채움
                df.at[i, 'deposit'] = same_area_rows['deposit'].dropna().iloc[0] * 1.05
            else:
                # 4. 같은 면적이 없으면 해당 그룹의 deposit 평균값으로 채움
                df.at[i, 'deposit'] = group_df['deposit'].mean()

    return df

In [47]:
df_filled = fill_deposit_na(concat_df)
df_filled

,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,deposit,apt_idx,area,area_price,area_m2_price,_type
0,0,84.9981,201906,25,2,9,2019,37.054314,127.045216,0,21525.0,0,25.8,658.914729,200.004471,train
1,1,84.9981,202003,26,2,20,2019,37.054314,127.045216,1,5250.0,0,25.8,891.472868,270.594284,train
2,2,84.9981,202003,28,2,8,2019,37.054314,127.045216,1,4200.0,0,25.8,891.472868,270.594284,train
3,3,59.3400,201907,15,2,1,1986,36.964647,127.055847,33,4200.0,1,18.0,277.777778,84.260195,train
4,4,59.8100,201904,12,2,6,1995,36.972390,127.084514,24,4200.0,2,18.1,99.447514,30.095302,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150167,150167,115.5101,202402,27,0,17,2010,37.528394,126.659398,14,30450.0,17869,35.0,NaN,NaN,test
150168,150168,142.8738,202403,2,0,4,2010,37.528394,126.659398,14,32550.0,17869,43.3,NaN,NaN,test
150169,150169,142.8738,202403,16,1,13,2010,37.528394,126.659398,14,32550.0,17869,43.3,NaN,NaN,test
150170,150170,114.9285,202403,22,1,2,2010,37.528394,126.659398,14,31500.0,17869,34.8,NaN,NaN,test


In [49]:
df_filled['deposit'].isnull().sum()

3398

In [54]:
train_data['deposit'].isnull().sum()

0

In [56]:
df_filled[df_filled['index']==411]

,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,deposit,apt_idx,area,area_price,area_m2_price,_type
411,411,59.87,201911,5,2,8,1999,36.980536,126.917535,20,NaN,21,18.1,497.237569,150.325706,train
411,411,59.79,202405,15,0,6,2024,37.585373,127.216700,0,NaN,18491,18.1,NaN,NaN,test


In [57]:
concat_df[concat_df['index']==411]

,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,deposit,apt_idx,area,area_price,area_m2_price,_type
411,411,59.87,201911,5,2,8,1999,36.980536,126.917535,20,NaN,21,18.1,497.237569,150.325706,train
411,411,59.79,202405,15,0,6,2024,37.585373,127.216700,0,NaN,18491,18.1,NaN,NaN,test


In [50]:
nan_deposit_rows = df_filled[df_filled['deposit'].isna()]
nan_deposit_rows

,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,deposit,apt_idx,area,area_price,area_m2_price,_type
411,411,59.870,201911,5,2,8,1999,36.980536,126.917535,20,NaN,21,18.1,497.237569,150.325706,train
424,424,84.996,201912,4,2,14,2004,36.987779,126.919841,15,NaN,20,25.8,581.395349,176.478893,train
426,426,84.996,201912,9,2,11,2004,36.987779,126.919841,15,NaN,20,25.8,600.775194,182.361523,train
428,428,59.870,201912,15,2,13,1999,36.980536,126.917535,20,NaN,21,18.1,524.861878,158.677134,train
429,429,59.870,201912,15,2,13,1999,36.980536,126.917535,20,NaN,21,18.1,524.861878,158.677134,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142684,142684,84.830,202403,26,0,18,2024,37.490141,126.503261,0,NaN,18616,25.7,NaN,NaN,test
142914,142914,75.620,202401,16,0,3,2004,37.463234,126.642899,20,NaN,18673,22.9,NaN,NaN,test
142915,142915,75.620,202401,16,0,3,2004,37.463234,126.642899,20,NaN,18673,22.9,NaN,NaN,test
143189,143189,49.900,202403,25,0,13,2015,37.459061,126.674991,9,NaN,18674,15.1,NaN,NaN,test


In [40]:
train_df = concat_df[concat_df['_type']=="train"]
test_df = concat_df[concat_df['_type']=="test"]

In [41]:
columns_needed = ['area_m2', 'contract_year_month', 'contract_day', 'contract_type', 'floor', 'latitude', 'longitude',
                  'deposit', 'apt_idx']
columns_needed_test = ['area_m2', 'contract_year_month', 'contract_day', 'contract_type', 'floor', 'latitude',
                       'longitude','apt_idx']
train_data = train_df[columns_needed]
test_data = test_df[columns_needed_test]

# Holdout 데이터셋 설정 (예: 2023년 7월부터 12월까지의 데이터)

In [42]:
holdout_start = 202307
holdout_end = 202312
holdout_data = train_data[
    (train_data['contract_year_month'] >= holdout_start) & (train_data['contract_year_month'] <= holdout_end)]
train_data = train_data[
    ~((train_data['contract_year_month'] >= holdout_start) & (train_data['contract_year_month'] <= holdout_end))]

# 학습 데이터와 정답 데이터 분리

In [43]:
X_train = train_data.drop(columns=['deposit'])
y_train = train_data['deposit']
X_holdout = holdout_data.drop(columns=['deposit'])
y_holdout = holdout_data['deposit']
X_test = test_data.copy()

## XGBoost 모델 훈련

In [44]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(random_state=RANDOM_SEED)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [45]:
# 홀드아웃 데이터에 대한 예측
xgb_holdout_pred = xgb_model.predict(X_holdout)

# 홀드아웃 데이터의 MAE 계산
from sklearn.metrics import mean_absolute_error
xgb_holdout_mae = mean_absolute_error(y_holdout, xgb_holdout_pred)

print("Holdout 데이터셋 성능:")
print(f"XGBoost MAE: {xgb_holdout_mae:.2f}")

Holdout 데이터셋 성능:
XGBoost MAE: 6489.70


In [18]:
xgb_test_pred = xgb_model.predict(X_test)
sample_submission['deposit'] = xgb_test_pred
sample_submission.to_csv('output.csv', index=False, encoding='utf-8-sig')

# LightGBM 모델 훈련

In [14]:
lgb_model = lgb.LGBMRegressor(random_state=RANDOM_SEED)
lgb_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010378 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 914
[LightGBM] [Info] Number of data points in the train set: 1594362, number of used features: 7
[LightGBM] [Info] Start training from score 37814.733261


LGBMRegressor(random_state=42)

# Holdout 데이터셋에 대한 성능 확인

In [15]:
lgb_holdout_pred = lgb_model.predict(X_holdout)
lgb_holdout_mae = mean_absolute_error(y_holdout, lgb_holdout_pred)
print("Holdout 데이터셋 성능:")
print(f"LightGBM MAE: {lgb_holdout_mae:.2f}")

Holdout 데이터셋 성능:
LightGBM MAE: 7904.04


# Sample Submission 제출하기

In [15]:
lgb_test_pred = lgb_model.predict(X_test)
sample_submission['deposit'] = lgb_test_pred
sample_submission.to_csv('output.csv', index=False, encoding='utf-8-sig')